<a href="https://colab.research.google.com/github/aquapathos/openCVLesson/blob/master/IP/%E6%98%9F%E7%A9%BA%E7%94%BB%E5%83%8F%E3%81%AE%EF%BC%92%E9%9A%8E%E8%AA%BF%E5%8C%96.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 判別分析法と移動平均法

　星空の画像を２階調化して星だけを白く浮かび上がらせたい。
 
　しきい値を自動決定する判別分析法（大津の方法）と近傍の明るさに応じてしきい値を画素ごとに設定する移動平均法を適用します。実験を通じて２つの方法を実践的に理解してもらうのが狙いです。
 
 ---
 # 準備 処理対象画像の読み込みとヒストグラムの確認

In [ ]:
import urllib.request
import cv2
from google.colab.patches import cv2_imshow
from matplotlib import pyplot as plt
from matplotlib import pylab
from google.colab import widgets
import warnings
import matplotlib
warnings.filterwarnings('ignore', category=matplotlib.MatplotlibDeprecationWarning)

WIDTH=400

url = "https://user-images.githubusercontent.com/5820803/111906119-62a87e80-8a92-11eb-8550-c704aff0cf26.png" # 画像URL
img_path, header = urllib.request.urlretrieve(url)
img = cv2.imread(img_path,0) #  画像の読み込み
width = WIDTH
height = img.shape[0]*WIDTH//img.shape[1]
img = cv2.resize(img,(width,height))
cv2_imshow(img) #
print("図2-１　処理対象画像")
cv2.imwrite("入力画像.png",img)
  
hist = cv2.calcHist([img],[0],None,[256],[0,256])
plt.hist(img.ravel(),256,[0,256]); plt.show()
print("図２-2　ヒストグラム")

## 実験１しきい値の手設定による２階調化

ヒストグラムを参照して、さまざまなしきい値を設定して２階調化を試みてみよ。

## 課題　結果の例を３つ報告せよ。

In [ ]:
#@title Number fields
しきい値１ = 24 #@param {type:"slider", min:0, max:255, step:1}
しきい値２ = 74 #@param {type:"slider", min:0, max:255, step:1}
しきい値３ = 144 #@param {type:"slider", min:0, max:255, step:1}
ths=[しきい値１,しきい値2,しきい値3]
  
for i in range(3):
    ret,bwimg = cv2.threshold(img,ths[i],255,cv2.THRESH_BINARY)
    plt.imshow(bwimg,'gray', vmin = 0, vmax = 255) 
    plt.axis('off')
    # plt.axes().set_aspect('equal')
    plt.show()
    print("          図2-{}     ２値化画像　　しきい値{}\n".format(i+3,ths[i]))

# 実験２　判別分析法（大津の方法）

　　判別分析法で２階調化のしきい値を決定した場合の結果はどうなるか試してみよう。
  
 ## 課題　判別分析法の結果は星空の２階調化に向いているかどうか、向いていないとしたら、どうするとよいと思うか、考察せよ。
 
 　　なお、下のプログラムでは、ret2 という変数の値が、判別分析法によって決定されたしきい値である。


In [ ]:
ret2,otsu = cv2.threshold(img,0,255,cv2.THRESH_OTSU)
plt.imshow(otsu,'gray', vmin = 0, vmax = 255) 
plt.axis('off')
# plt.axes().set_aspect('equal')
plt.show()
print("          図2-6     判別分析法の結果　　しきい値{}\n".format(ret2))


# 実験３　移動平均法

- ksize 　近傍の範囲　　デフォルト　11
- offset   オフセット値　デフォルト　-2

　近傍の平均値よりオフセット値分低いレベルをしきい値とし、それより明るければ白、暗ければ黒とする。

　文字などの場合は平均だと紙の白部分に少しでも暗い部分があると黒になってしまうので、平均より低い値をしきい値として白よりになるようにする。

　星空の場合は逆で、背景の黒をしっかり出したいので、オフセットを負とするとよい。

　平均の取り方としては単純平均と荷重平均があるが、ここでは**ガウシアン重みによる荷重平均**を用いた。
 
---
## 課題   オフセット値を変えてみて、結果の変化を調べ、もっとも星空っぽく見えると思う値に定めよ。

　　いうまでもないが、オフセットの値と結果画像を報告せよ。他の課題も同様。

In [ ]:
#@title Number fields
offset = 0 #@param {type:"slider", min:-30, max:20, step:1}
ksize = 11 #@param {type:"slider", min:3, max:20, step:2}
th3 = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
            cv2.THRESH_BINARY,ksize,offset)
plt.imshow(th3,'gray', vmin = 0, vmax = 255) 
plt.axis('off')
# plt.axes().set_aspect('equal')
plt.show()

print("          図2-7     移動平均法の結果　")


# 発展
　　樹木の輪郭部分の隙間が白く残るのがきになるのでなんとかしたい。
  
  　以下のプログラムでは、
   
 1. まず画像をぼかし、これに対して判別分析法を適用して目安となるしきい値を決定。
 2. 上で得たしきい値より暗いレベルで２階調化。夜空部分はほぼ白、樹木は黒となる。
 3. 白領域に残った黒画素を除去するためにオープニング処理を施す。
 4. 残った黒連結成分を膨張させ、マスク画像とする。
 5. 移動平均法の結果画像とマスク画像をビット毎に論理積をとる。
 
 という手順で空の星だけを白く残そうとしている。


In [ ]:
import numpy as np
blur = cv2.GaussianBlur(img,(5,5),0) # 画像をわざとボケさせる
kernel = np.ones((5,5),np.uint8)
ret3,th4 = cv2.threshold(blur,0,255,cv2.THRESH_OTSU) # 判別分析法で２値化のしきい値を決める。
ret,bwimg = cv2.threshold(img,ret3-30,255,cv2.THRESH_BINARY) # 上でえたしきい値より低いレベルで２値化
opening = cv2.morphologyEx(bwimg, cv2.MORPH_OPEN, kernel) # 白領域内の黒ノイズを除去
erosion = cv2.erode(opening,kernel,iterations = 5) # 黒領域を膨張させる（樹木の輪郭部分の隙間を埋めるため）
plt.imshow(erosion,'gray', vmin = 0, vmax = 255) 
plt.axis('off')
# plt.axes().set_aspect('equal')
plt.show()
print("          図2-8     マスク画像\n")


## 最終結果　マスク画像と移動平均法の結果のAND演算結果

In [ ]:
plt.imshow(cv2.bitwise_and(th3,erosion),'gray', vmin = 0, vmax = 255) 
plt.axis('off')
# plt.axes().set_aspect('equal')
plt.show()
print("          図2-9    星のみを白画素として抽出した画像\n")

# 課題

　上の結果から星の数を自動カウントする方法を考えなさいと言われ、  
 　O君は、「白画素の数を数えればよい」と解答した。  
　彼の考え方には問題がある。その理由と、どう答えるべきかを論ぜよ。

